In [1]:
import pandas as pd
import numpy as np
import os
import importlib
import external_func
importlib.reload(external_func)

from external_func import *

folder = 'data'
nazwy_plikow = pobierz_nazwy_plikow(folder)
print(nazwy_plikow)
pliki = {}
for nazwa_pliku in nazwy_plikow:
    wczytaj_dane(folder, nazwa_pliku, pliki)

['title.ratings.tsv', 'title.principals.tsv', 'title.akas.tsv', 'name.basics.tsv', 'title.basics.tsv', 'title.episode.tsv', 'title.crew.tsv']


/Users/machoranis/Programowanie/NYPD/NYPD-Final_project/external_func.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv(pelna_nazwa, sep='\t')


In [2]:
klucze = [s[:-4].replace('.', '_') for s in nazwy_plikow]
klucze

['title_ratings',
 'title_principals',
 'title_akas',
 'name_basics',
 'title_basics',
 'title_episode',
 'title_crew']

In [3]:
for i in range(len(klucze)):
    print(pliki[klucze[i]].columns)

Index(['tconst', 'averageRating', 'numVotes'], dtype='object')
Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')
Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')
Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber'], dtype='object')
Index(['tconst', 'directors', 'writers'], dtype='object')


# tworzenie metryki
Uznałem, ze najlepszą metryką będzie średnia harmoniczna z averageRating * log(numVotes), poniewaz srednia harmoniczna jest zawsze nizsza od sredniej arytmetycznej i dla stałej średniej arytmetyczniej osiąga maksimum gdy wszystkie oceny są równe, więc faworyzujemy niskie odchylenia. Z kolei log(numVotes), poniewaz wydaje się, ze jest to podobny poziom dla
tego samego rzędu wielkości, ale jednak faworyzuje trochę filmy popularniejsze. Np wezmy pod uwagę film ze srednią 9.1 i 100k głosów oraz film 8.9 z 130k głosów. Wówczas wg naszej metryki ocena będzie równa

In [4]:
pie = 9.1 * (np.log(100000))
dru = 8.9 * (np.log(130000))
print("Pierwszy ma ocenę ",pie, ", zaś drugi ", dru)

Pierwszy ma ocenę  104.76762173122907 , zaś drugi  104.80007859199571


Stąd widzimy, ze "ludzkim okiem" nasza metryka ma sens.

# Task 1

In [5]:
df = pliki[klucze[0]]

Dodajmy kolumnę quality, która liczy naszą metrykę dla kadego filmu

In [6]:
df['quality'] = df['averageRating'] * np.log(df['numVotes'])

In [7]:
df.head(5)

,tconst,averageRating,numVotes,quality
0,tt0000001,5.7,2063,43.501924
1,tt0000002,5.6,279,31.534786
2,tt0000003,6.5,2030,49.502642
3,tt0000004,5.4,180,28.041967
4,tt0000005,6.2,2796,49.202860


In [8]:
df2 = pliki[klucze[4]]
df2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
df3 = df.merge(df2, left_on='tconst', right_on='tconst', how='left')

In [10]:
df3['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame'], dtype=object)

In [11]:
df3 = df3[df3['titleType'].isin(['movie', 'tvMovie'])]
df3.sort_values('quality', ascending=False).head(20)

,tconst,averageRating,numVotes,quality,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
83943,tt0111161,9.3,2912371,138.425646,movie,The Shawshank Redemption,The Shawshank Redemption,0,1994,\N,142,Drama
255737,tt0468569,9.0,2893597,133.902098,movie,The Dark Knight,The Dark Knight,0,2008,\N,152,"Action,Crime,Drama"
47432,tt0068646,9.2,2029452,133.614143,movie,The Godfather,The Godfather,0,1972,\N,175,"Crime,Drama"
116492,tt0167260,9.0,1994127,130.551452,movie,The Lord of the Rings: The Return of the King,The Lord of the Rings: The Return of the King,0,2003,\N,201,"Action,Adventure,Drama"
83726,tt0110912,8.9,2238375,130.129220,movie,Pulp Fiction,Pulp Fiction,0,1994,\N,154,"Crime,Drama"
665929,tt1375666,8.8,2570529,129.884676,movie,Inception,Inception,0,2010,\N,148,"Action,Adventure,Sci-Fi"
91757,tt0120737,8.9,2022740,129.227676,movie,The Lord of the Rings: The Fellowship of the Ring,The Lord of the Rings: The Fellowship of the Ring,0,2001,\N,178,"Action,Adventure,Drama"
100688,tt0137523,8.8,2345337,129.077869,movie,Fight Club,Fight Club,0,1999,\N,139,Drama
82844,tt0109830,8.8,2277257,128.818643,movie,Forrest Gump,Forrest Gump,0,1994,\N,142,"Drama,Romance"
81342,tt0108052,9.0,1462031,127.758034,movie,Schindler's List,Schindler's List,0,1993,\N,195,"Biography,Drama,History"


In [20]:
tmp = pliki[klucze[2]]
tmp_ot = tmp[tmp['isOriginalTitle'] == 1] #originaltitle
tmp2 = tmp.merge(tmp_ot, left_on='titleId', right_on='titleId', how='left')
tmp2.head()

,titleId,ordering_x,title_x,region_x,language_x,types_x,attributes_x,isOriginalTitle_x,ordering_y,title_y,region_y,language_y,types_y,attributes_y,isOriginalTitle_y
0,tt0000001,1,Carmencita,\N,\N,original,\N,1,1.0,Carmencita,\N,\N,original,\N,1.0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0,1.0,Carmencita,\N,\N,original,\N,1.0
2,tt0000001,3,Carmencita,US,\N,imdbDisplay,\N,0,1.0,Carmencita,\N,\N,original,\N,1.0
3,tt0000001,4,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0,1.0,Carmencita,\N,\N,original,\N,1.0
4,tt0000001,5,Καρμενσίτα,GR,\N,imdbDisplay,\N,0,1.0,Carmencita,\N,\N,original,\N,1.0


In [23]:
tmp3 = tmp2[tmp2['title_x'] == tmp2['title_y']]
tmp3.info

221933205

In [30]:
tmp3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14795547 entries, 0 to 48887169
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   titleId            object 
 1   ordering_x         int64  
 2   title_x            object 
 3   region_x           object 
 4   language_x         object 
 5   types_x            object 
 6   attributes_x       object 
 7   isOriginalTitle_x  int64  
 8   ordering_y         float64
 9   title_y            object 
 10  region_y           object 
 11  language_y         object 
 12  types_y            object 
 13  attributes_y       object 
 14  isOriginalTitle_y  float64
dtypes: float64(2), int64(2), object(11)
memory usage: 1.8+ GB


In [29]:
tmp4 = tmp3[tmp3['region_x'] != '\\N']
tmp4 = tmp4[['titleId', 'region_x']]
tmp4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3913878 entries, 1 to 48887168
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   titleId   object
 1   region_x  object
dtypes: object(2)
memory usage: 89.6+ MB


In [38]:
df4 = df3.merge(tmp4, left_on='tconst', right_on='titleId', how='left')
df4 = df4.rename(columns={'region_x': 'region'})
df4 = df4.sort_values('quality', ascending=False)

In [39]:
# Group by 'title' and 'quality', and aggregate 'region' into a list
grouped_df = df4.groupby(['tconst', 'primaryTitle', 'quality'])['region'].agg(list).reset_index()

# Print the resulting DataFrame

print(grouped_df)

           tconst                   primaryTitle    quality            region
0       tt0000009                     Miss Jerry  28.925566      [AU, HU, US]
1       tt0000147  The Corbett-Fitzsimmons Fight  32.489823              [US]
2       tt0000502                       Bohemios  12.139561              [ES]
3       tt0000574    The Story of the Kelly Gang  40.906983  [US, SG, GB, AU]
4       tt0000591               The Prodigal Son  17.919531              [FR]
...           ...                            ...        ...               ...
366084  tt9916428            The Secret of China  10.599980             [nan]
366085  tt9916460                      Pink Taxi  26.205507             [nan]
366086  tt9916538            Kuambil Lagi Hatiku  18.236964              [ID]
366087  tt9916706                Dankyavar Danka  12.231728              [IN]
366088  tt9916730                         6 Gunn  17.394347             [nan]

[366089 rows x 4 columns]


In [45]:
uk_count = grouped_df['region'].apply(lambda x: x.count('')).sum()

print(f"'nan' appears {uk_count} times in the 'country' column.")

'nan' appears 0 times in the 'country' column.
